In [11]:
from google.colab import drive

# from PIL import Image
import cv2
import numpy as np

In [12]:
class driveFile:
  def __init__(self, mountpath):
    import os
    self.path = mountpath
    drive.mount(self.path)
  def dataload(self,folder_path):
    img=[]
    label=[]

    self.path1=os.path.join(self.path,folder_path)

    if os.path.exists(self.path1):
      for i in os.listdir(self.path1):
        self.dis_path=os.path.join(self.path1,i)
        # /content/drive/MyDrive/train_set/BA- cellulitis
        if os.path.exists(self.dis_path) and '.DS_Store' not in self.dis_path:
          for j in os.listdir(self.dis_path):
           if j.endswith(('.jpg', '.png', '.jpeg')):
              self.img_path = os.path.join(self.dis_path, j)
              loaded_img= self.imgload(self.img_path)
              if loaded_img is not None:
                img.append(loaded_img)
                label.append(i)
              else:
                 print("Skipping non-image file:", j)
          else:
              print("Skipping invalid path or .DS_Store:", self.dis_path)
              continue
      return np.array(img), np.array(label)
    else:
      return "patherror","patherror"



  def imgload(self,imgpath):
    img=cv2.imread(imgpath)
    if img is None:
      print(f"Failed to load image at {imgpath}")
      return None
    img=cv2.resize(img,(20,20))
    # v imp, usually 224*224 but here 20*20 "downsampling or using a smaller image resolution before flattening it."
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=img.astype('float32')/255.0
    img_array=np.array(img)
    return img_array


In [13]:
drive=driveFile('/content/drive')
ti,tl=drive.dataload('MyDrive/train_set')
# drive.dataload('')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Skipping invalid path or .DS_Store: /content/drive/MyDrive/train_set/BA- cellulitis
Skipping invalid path or .DS_Store: /content/drive/MyDrive/train_set/FU-athlete-foot


In [14]:
print(ti.shape)
# 321 images, 244*244 pixels grayscale
print(ti[0].shape)
# 224*224
ti_flat = np.array([img.flatten() for img in ti])
# The perceptron (and most neural networks) operates on vectors, not matrices.
print(ti_flat[0].shape)

(197, 20, 20)
(20, 20)
(400,)


In [15]:
import pandas as pd
df = pd.DataFrame({
    'image': ti_flat.tolist(),  # Image data (NumPy arrays)
    'label': tl   # Corresponding labels
})

disease_counts = df['label'].value_counts()
print(disease_counts)
df=pd.concat([df,pd.get_dummies(df['label'])],axis=1).replace([True,False],[1,0]).drop('label',axis=1)
df.tail()

label
FU-athlete-foot    101
BA- cellulitis      96
Name: count, dtype: int64


<ipython-input-15-3a8501e5a8ea>:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df=pd.concat([df,pd.get_dummies(df['label'])],axis=1).replace([True,False],[1,0]).drop('label',axis=1)


,image,BA- cellulitis,FU-athlete-foot
192,"[0.47843137383461, 0.6000000238418579, 0.63529...",0,1
193,"[1.0, 1.0, 0.9960784316062927, 0.9843137264251...",0,1
194,"[0.9450980424880981, 0.9450980424880981, 0.937...",0,1
195,"[0.6666666865348816, 0.4470588266849518, 0.431...",0,1
196,"[0.8941176533699036, 0.9254902005195618, 0.917...",0,1


In [16]:
def perceptron(x,w,b):
  y_linearop=np.dot(x.T,w)+b
  #unbounded and doesn't provide much information in terms of probabilities.
  return y_linearop
#activatn fn
def sigmoid(y_linearop):
  y_predprob=1/(1+np.exp(-y_linearop))
  return y_predprob
def binary_cross_entropy(y,y_pred):
  epsilon=1e-15
  # epsilon small as such y_pred=0-->y=epsilon,y_pred=1-->1-eps
  y_pred=np.clip(y_pred,epsilon,1-epsilon)
  # y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
  loss=-(y*np.log(y_pred))+((1-y)*np.log(1-y_pred))
  return loss
def backward_prop(x,w,b,y_linearop,y,y_pred):
  dloss_dy_pred=-(y/y_pred)-((1-y)/(1-y_pred))
  dsigm_dlinearoutput=y_pred*(1-y_pred)
  # dsigmoid/dx=sigmod-sigmoid^2
  dloss_dlinearoutput=dloss_dy_pred*dsigm_dlinearoutput
  # (f(g(x)))`=f`(g(x))g'(x)
  dloss_dw=dsigm_dlinearoutput*x
  dloss_db=dsigm_dlinearoutput*1
  return [dloss_dw,dloss_db]
def update_para(w,b,diff,learning_rate):
  w -= learning_rate * diff[0]
  b -= learning_rate * diff[1]
  return w,b

In [17]:
def dl(x,w,b,y,learningrate):
  y_pred=sigmoid(perceptron(x,w,b))
  loss=binary_cross_entropy(y,y_pred)
  w,b=update_para(w,b,backward_prop(x,w,b,perceptron(x,w,b),y,y_pred),learningrate)
  print(loss)

In [20]:
w=np.random.randn(400,1)
# (i\p features,o\p classes)
dl(ti_flat[0].reshape(-1,1),w,0,1,0.0001)
# b = np.zeros((1, 3))


[[15.63189689]]


In [ ]:
# sampled_df = df.groupby('label').apply(lambda x: x.sample(n=20, random_state=42) if len(x) > 20 else x)
# sampled_df = sampled_df.reset_index(drop=True)

In [ ]:
# images20=np.array(sampled_df['image'])
# images20.size

In [ ]:
# def sigmoid(x):
#     return 1 / (1 + np.exp(-x))
# def forward_propagation(X, W, b):
#     Z = np.dot(X, W) + b
#     A = sigmoid(Z)
#     return A  # A is the output of the sigmoid activation (probabilities)
# def compute_cost(A, Y):
#     m = Y.shape[0]  # Number of training examples
#     # Cross-entropy cost
#     cost = -np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A)) / m
#     return cost
